In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')

In [ ]:
train.head(10)

In [ ]:
train.shape

In [ ]:
test.head(10)

In [ ]:
test.shape

In [ ]:
train.describe()

In [ ]:
test.describe()

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
train["Survived"].value_counts()

In [ ]:
g = sns.FacetGrid(train,col ="Survived",row = "Pclass",hue = "Sex",height = 6)
g.map(sns.kdeplot,"Fare",shade=True)
g.add_legend()
plt.xlim(0,100)

In [ ]:
train_copy = train.copy()

In [ ]:
test_copy = test.copy()

In [ ]:
train_test_data = [train_copy, test_copy]

In [ ]:
for dataset in train_test_data:
    dataset['Title'] = dataset['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
train_copy['Title'].value_counts()

In [ ]:
test_copy["Title"].value_counts()

In [ ]:
for dataset in train_test_data:
    dataset['Title'] = dataset['Title'].replace(['Dona','Lady', 'Countess','Capt', 'Col','Don','Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Mlle','Mme','Ms'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Miss', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mr', 'Mr')
    dataset['Title'] = dataset['Title'].replace('Mrs', 'Mrs')
    dataset['Title'] = dataset['Title'].replace('Master', 'Master')

In [ ]:
title_mapping = {"Rare": 1, "Miss": 2, "Mr": 3, "Mrs": 4, "Master": 5}
for dataset in train_test_data:
    dataset['Title'] = dataset['Title'].map(title_mapping)

In [ ]:
train_copy.head()

In [ ]:
train_copy["Age"].fillna(train_copy.groupby("Title")["Age"].transform("median"), inplace=True)
test_copy["Age"].fillna(test_copy.groupby("Title")["Age"].transform("median"), inplace=True)

In [ ]:
train_copy.head()

In [ ]:
train_copy["Age"].isnull().sum()

In [ ]:
train_copy["Fare"].fillna(train_copy.groupby("Pclass")["Fare"].transform("median"), inplace=True)
test_copy["Fare"].fillna(test_copy.groupby("Pclass")["Fare"].transform("median"), inplace=True)

In [ ]:
train_copy["Fare"].isnull().sum()

In [ ]:
train_copy["Embarked"].value_counts()

In [ ]:
test_copy["Embarked"].value_counts()

In [ ]:
train_copy["Embarked"].fillna("S",inplace = True)
test_copy["Embarked"].fillna("S",inplace = True)

In [ ]:
train_copy["FamilySize"] = train_copy["SibSp"] + train_copy["Parch"] + 1
test_copy["FamilySize"] = test_copy["SibSp"] + test_copy["Parch"] + 1

In [ ]:
for dataset in train_test_data:
    dataset['Age_bin'] = pd.cut(dataset['Age'], bins=[0,10,20,30,40,60,80], labels=['Children','Teenage','Boy','Adult','Man','Elder'])

In [ ]:
age_mapping = {'Children': 1,'Teenage': 2,'Boy': 3,'Adult': 4,'Man': 5,'Elder': 6}
for dataset in train_test_data:
    dataset['Age_bin'] = dataset['Age_bin'].map(age_mapping)

In [ ]:
train_test_data[0].head()

In [ ]:
for dataset in train_test_data:
    dataset['Fare_bin'] = pd.cut(dataset['Fare'], bins=[-1,7,14,28,40,120,600], labels=['Low_fare','mild_fare','medium_fare',
                                                                                      'Average_fare','high_fare','luxury_fare'])

In [ ]:
fare_mapping = {'Low_fare': 1,'mild_fare': 2,'medium_fare': 3,'Average_fare': 4,'high_fare': 5,'luxury_fare': 6}
for dataset in train_test_data:
    dataset['Fare_bin'] = dataset['Fare_bin'].map(fare_mapping)

In [ ]:
train_copy["Fare_bin"].isnull().sum()

In [ ]:
train_copy["Cabin"].value_counts()

In [ ]:
for dataset in train_test_data:
    dataset['Cabin'] = dataset['Cabin'].str[:1]

In [ ]:
train_copy.head()

In [ ]:
cabin_mapping = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "T": 8}
for dataset in train_test_data:
    dataset['Cabin'] = dataset['Cabin'].map(cabin_mapping)

In [ ]:
train_copy["Cabin"].fillna(train_copy.groupby("Pclass")["Cabin"].transform("median"), inplace=True)
test_copy["Cabin"].fillna(test_copy.groupby("Pclass")["Cabin"].transform("median"), inplace=True)

In [ ]:
train_copy.head()

In [ ]:
sex_mapping = {"male": 1, "female": 2}
for dataset in train_test_data:
    dataset["Sex"] = dataset["Sex"].map(sex_mapping)

In [ ]:
embarked_mapping = {"S": 1, "C": 2, "Q": 3}
for dataset in train_test_data:
    dataset["Embarked"] = dataset["Embarked"].map(embarked_mapping)

In [ ]:
train_copy.head()

In [ ]:
train_copy.isnull().sum()

In [ ]:
test_copy.isnull().sum()

In [ ]:
Train = train_copy.drop(["Name","Ticket","Age","Fare"],axis = 1)
Test = test_copy.drop(["Name","Ticket","Age","Fare"],axis = 1)

In [ ]:
Test.head()

In [ ]:
Train.head()

In [ ]:
train_x = Train.drop(["PassengerId","Survived"],axis = 1)
train_y = Train["Survived"]
test_x = Test.drop(["PassengerId"],axis = 1)

In [ ]:
train_x.head()

In [ ]:
train_x.shape

In [ ]:
test_x.head()

In [ ]:
test_x.shape

In [ ]:
train_y.head()

In [ ]:
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

In [ ]:
cv_split = model_selection.ShuffleSplit(n_splits = 10, test_size = .3, train_size = .6, random_state = 0 )

In [ ]:
dtree = tree.DecisionTreeClassifier(random_state = 0)
base_results = model_selection.cross_validate(dtree, train_x, train_y, cv  = cv_split)
dtree.fit(train_x, train_y)

print('BEFORE Choosing Decision Tree Parameters: ', dtree.get_params())
#print("BEFORE DT Training w/bin score mean: {:.2f}". format(base_results['train_score'].mean()*100)) 
print("BEFORE DT Test w/bin score mean: {:.2f}". format(base_results['test_score'].mean()*100))
print("BEFORE DT Test w/bin score 3*std: +/- {:.2f}". format(base_results['test_score'].std()*100*3))
#print("BEFORE DT Test w/bin set score min: {:.2f}". format(base_results['test_score'].min()*100))
print('-'*10)

In [ ]:
base_results

In [ ]:
param_grid = {'criterion': ['gini', 'entropy'],'max_depth': [2,4,6,8,10,None],'random_state': [0]}

In [ ]:
dtree_feat = feature_selection.RFECV(dtree, step = 1, scoring = 'accuracy', cv = cv_split)
dtree_feat.fit(train_x, train_y)

In [ ]:
X_feat = train_x.columns.values[dtree_feat.get_support()]
X_feat

In [ ]:
train_x[X_feat]

In [ ]:
feat_results = model_selection.cross_validate(dtree, train_x[X_feat], train_y, cv  = cv_split)

In [ ]:
print('AFTER Feature selection of Decision Tree Training Shape New: ', train_x[X_feat].shape) 
print('AFTER DT RFE Training Columns New: ', X_feat)

#print("AFTER DT RFE Training w/bin score mean: {:.2f}". format(feat_results['train_score'].mean()*100)) 
print("AFTER DT RFE Test w/bin score mean: {:.2f}". format(feat_results['test_score'].mean()*100))
print("AFTER DT RFE Test w/bin score 3*std: +/- {:.2f}". format(feat_results['test_score'].std()*100*3))
print('-'*10)

In [ ]:
feat_tune_model = model_selection.GridSearchCV(tree.DecisionTreeClassifier(), param_grid=param_grid, scoring = 'roc_auc', cv = cv_split)
feat_tune_model.fit(train_x[X_feat], train_y)

In [ ]:
print('AFTER  Decision Tree Feature Selection Tuned Parameters: ', feat_tune_model.best_params_)

In [ ]:
dtree1 = tree.DecisionTreeClassifier(criterion = "gini", max_depth = 4, random_state = 0)
base_results = model_selection.cross_validate(dtree1, train_x[X_feat], train_y, cv  = cv_split)
dtree1.fit(train_x[X_feat], train_y)

In [ ]:
print("BEFORE DT Test w/bin score mean: {:.2f}". format(base_results['test_score'].mean()*100))
print("BEFORE DT Test w/bin score 3*std: +/- {:.2f}". format(base_results['test_score'].std()*100*3))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
randf = RandomForestClassifier(n_estimators=2500, max_depth=4)
base_results1 = model_selection.cross_validate(randf, train_x[X_feat], train_y, cv  = cv_split)

In [ ]:
print("BEFORE Random Forrest Test w/bin score mean: {:.2f}". format(base_results1['test_score'].mean()*100))
print("BEFORE Random Forrest Test w/bin score 3*std: +/- {:.2f}". format(base_results1['test_score'].std()*100*3))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

base_results2 = model_selection.cross_validate(knn, train_x[X_feat], train_y, cv  = cv_split)

In [ ]:
print("BEFORE KNN Test w/bin score mean: {:.2f}". format(base_results2['test_score'].mean()*100))
print("BEFORE KNN Test w/bin score 3*std: +/- {:.2f}". format(base_results2['test_score'].std()*100*3))

In [ ]:
from sklearn.svm import SVC
sv = SVC()
base_results3 = model_selection.cross_validate(sv, train_x[X_feat], train_y, cv  = cv_split)
sv.fit(train_x[X_feat], train_y)

In [ ]:
print("BEFORE SVM Test w/bin score mean: {:.2f}". format(base_results3['test_score'].mean()*100))
print("BEFORE SVM Test w/bin score 3*std: +/- {:.2f}". format(base_results3['test_score'].std()*100*3))

In [ ]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve

In [ ]:
# Cross validate model with Kfold stratified cross val
kfold = StratifiedKFold(n_splits=10)

In [ ]:
# Adaboost
DTC = DecisionTreeClassifier()

adaDTC = AdaBoostClassifier(DTC, random_state=7)

ada_param_grid = {"base_estimator__criterion" : ["gini", "entropy"],
              "base_estimator__splitter" :   ["best", "random"],
              "algorithm" : ["SAMME","SAMME.R"],
              "n_estimators" :[1,2],
              "learning_rate":  [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3,1.5]}

gsadaDTC = GridSearchCV(adaDTC,param_grid = ada_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsadaDTC.fit(train_x[X_feat],train_y)

ada_best = gsadaDTC.best_estimator_
gsadaDTC.best_score_

In [ ]:
#ExtraTrees 
ExtC = ExtraTreesClassifier()


## Search grid for optimal parameters
ex_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 8],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}


gsExtC = GridSearchCV(ExtC,param_grid = ex_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsExtC.fit(train_x[X_feat],train_y)

ExtC_best = gsExtC.best_estimator_

# Best score
gsExtC.best_score_

In [ ]:
# RFC Parameters tunning 
RFC = RandomForestClassifier()


## Search grid for optimal parameters
rf_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 8],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}


gsRFC = GridSearchCV(RFC,param_grid = rf_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsRFC.fit(train_x[X_feat],train_y)

RFC_best = gsRFC.best_estimator_

# Best score
gsRFC.best_score_

In [ ]:
# Gradient boosting tunning

GBC = GradientBoostingClassifier()
gb_param_grid = {'loss' : ["deviance"],
              'n_estimators' : [100,200,300],
              'learning_rate': [0.1, 0.05, 0.01],
              'max_depth': [4, 8],
              'min_samples_leaf': [100,150],
              'max_features': [0.3, 0.1] 
              }

gsGBC = GridSearchCV(GBC,param_grid = gb_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsGBC.fit(train_x[X_feat],train_y)

GBC_best = gsGBC.best_estimator_

# Best score
gsGBC.best_score_

In [ ]:
### SVC classifier
SVMC = SVC(probability=True)
svc_param_grid = {'kernel': ['rbf'], 
                  'gamma': [ 0.001, 0.01, 0.1, 1],
                  'C': [1, 10, 50, 100,200,300, 1000]}

gsSVMC = GridSearchCV(SVMC,param_grid = svc_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsSVMC.fit(train_x[X_feat],train_y)

SVMC_best = gsSVMC.best_estimator_

# Best score
gsSVMC.best_score_

In [ ]:
votingC = VotingClassifier(estimators=[('rfc', RFC_best), ('extc', ExtC_best),
('svc', SVMC_best), ('adac',ada_best),('gbc',GBC_best)], voting='soft', n_jobs=4)

votingC = votingC.fit(train_x[X_feat], train_y)

In [ ]:
survive = votingC.predict(test_x[X_feat])

In [ ]:
#survive = sv.predict(test_x[X_feat])

In [ ]:
survive

In [ ]:
survived = test["Pclass"] * 0

In [ ]:
survived

In [ ]:
for i in range(0,418):
    if survive[i] == 1:
        survived[i] = 1

In [ ]:
survived

In [ ]:
survived.value_counts()

In [ ]:
submission = pd.DataFrame({"PassengerId": test["PassengerId"],"Survived": survived})
submission.to_csv('62thsubmission.csv', index=False)

In [ ]:
submission_out = pd.read_csv('62thsubmission.csv')
submission_out.head(20)